# Machine Learning Project - Part D
## Ομάδα 1: Πλήρες Αυτόνομο Pipeline

**Φοιτητής:** Ευάγγελος Μόσχου  
**ΑΕΜ:** 10986

---

### Περιγραφή
Αυτό το notebook υλοποιεί το **Τελικό Μοντέλο** για την πρόκληση ταξινόμησης του Part D.

### Αρχιτεκτονική Συστήματος
```
┌─────────────────────────────────────────────────────────────┐
│                      ΤΕΛΙΚΟ ΜΟΝΤΕΛΟ                         │
├─────────────────────────────────────────────────────────────┤
│  1. Feature Selection (CatBoost/XGBoost importance)         │
│  2. Views: raw, quantile, pca, ica                          │
│  3. Models: XGBoost, CatBoost, TrueTabR, TabPFN             │
│  4. Stacking: LogisticRegression Meta-Learner               │
│  5. Self-Training: Pseudo-labeling με 95% confidence        │
└─────────────────────────────────────────────────────────────┘
```

### Τεχνολογίες
- **XGBoost DART**: Gradient boosting με dropout regularization
- **CatBoost Langevin**: SGLD για καλύτερη γενίκευση
- **TrueTabR**: Retrieval-based neural network με attention
- **TabPFN v2.5**: Foundation model με Post-Hoc Ensembling

**Τελευταία ενημέρωση:** 2026-01-13

## 1. Εγκατάσταση Dependencies
            
**Σκοπός:** Εγκατάσταση όλων των απαραίτητων βιβλιοθηκών.

**Βιβλιοθήκες:**
- `numpy`, `pandas`: Επεξεργασία δεδομένων
- `scikit-learn`: Preprocessing, CV, Meta-learning
- `torch`: Deep learning framework
- `xgboost`, `catboost`: Gradient boosting
- `tabpfn`, `tabpfn-extensions`: Foundation model για tabular data

> **ΠΡΟΣΟΧΗ (License):** Η έκδοση TabPFN v2.5 απαιτεί αποδοχή άδειας χρήσης (license agreement) κατά την εγκατάσταση/χρήση.

- `networkx`: Graph-based features (PageRank)

In [ ]:
# Εκτελέστε αυτό το κελί ΜΟΝΟ αν χρειάζεται εγκατάσταση
# !pip install numpy pandas scikit-learn torch xgboost catboost tabpfn tabpfn-extensions networkx scipy
# ΠΡΟΣΟΧΗ: Το TabPFN v2.5 απαιτεί license agreement (commercial/academic use)


## 2. Configuration (Ρυθμίσεις)

**Σκοπός:** Ορισμός όλων των hyperparameters και global settings.

**Κύριες Ρυθμίσεις:**
- `SEEDS`: Random seeds για reproducibility (5 seeds × averaging)
- `VIEWS`: Διαφορετικές αναπαραστάσεις δεδομένων (raw, quantile, pca, ica)
- `ENABLE_SELF_TRAIN`: Ενεργοποίηση pseudo-labeling
- `USE_STACKING`: Cross-fit stacking με meta-learner
- `TABPFN_MAX_TIME`: Χρονικό όριο για TabPFN PHE (60 sec)

In [ ]:
import os
import warnings
import random
import copy
from dataclasses import dataclass

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.decomposition import PCA, FastICA
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.manifold import SpectralEmbedding
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import NearestNeighbors, kneighbors_graph
from sklearn.preprocessing import QuantileTransformer, LabelEncoder
from sklearn.random_projection import GaussianRandomProjection
from scipy.optimize import nnls
from scipy.special import erfinv

warnings.filterwarnings('ignore')

# ============== CONFIGURATION ==============
class Config:
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    SEEDS = [1337, 1338, 1339, 1340, 1341]
    BATCH_SIZE = 512
    LR_SCALE = 2e-3
    SAM_RHO = 0.08
    TABM_K = 4
    DIFFUSION_EPOCHS = 30
    DAE_EPOCHS = 30
    DAE_NOISE_STD = 0.1
    GBDT_ITERATIONS = 500
    N_FOLDS = 5
    
    # Feature Engineering
    ALLOW_TRANSDUCTIVE = True
    MANIFOLD_K = 20
    ENABLE_PAGERANK = True
    ENABLE_LAPLACIAN = True
    USE_GPU_EIGENMAPS = True  # GPU acceleration for Laplacian
    ENABLE_DIFFUSION = True
    DIFFUSION_N_SAMPLES = 1000
    
    # Stacking
    USE_STACKING = True
    META_LEARNER = 'lr'  # lr | lgbm | ridge | nnls
    
    # TabPFN
    USE_TABPFN = True
    TABPFN_N_ENSEMBLES = 64
    TABPFN_MAX_TIME = 60
    
    # Self-Training
    ENABLE_SELF_TRAIN = True
    SELF_TRAIN_ITERS = 1
    SELF_TRAIN_CONF = 0.95
    SELF_TRAIN_AGREE = 1.0
    SELF_TRAIN_VIEW_AGREE = 0.66
    SELF_TRAIN_MAX = 10000
    SELF_TRAIN_WEIGHT_POWER = 1.0
    
    # Loss & Training
    LOSS_NAME = 'ce'
    LABEL_SMOOTHING = 0.0
    FOCAL_GAMMA = 2.0
    USE_CLASS_BALANCED = False
    CB_BETA = 0.999
    USE_MIXUP = True
    
    # Optional Features
    ENABLE_CORAL = False
    CORAL_REG = 1e-3
    ENABLE_ADV_REWEIGHT = False
    ENABLE_SWA = False
    SWA_START_EPOCH = 10
    ENABLE_TTT = False
    ENABLE_LID_SCALING = False
    
    # Checkpointing
    SAVE_CHECKPOINTS = True
    LOAD_CHECKPOINTS = False

config = Config()
VIEWS = ['raw', 'quantile', 'pca', 'ica']

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

print(f"Device: {config.DEVICE}")
print(f"Seeds: {config.SEEDS}")
print(f"Self-Training: {config.ENABLE_SELF_TRAIN} (iters={config.SELF_TRAIN_ITERS}, conf={config.SELF_TRAIN_CONF})")
print(f"GPU Eigenmaps: {config.USE_GPU_EIGENMAPS}")


## 3. Φόρτωση Δεδομένων

**Σκοπός:** Φόρτωση train/test datasets από CSV.

**Input:**
- `datasetTV.csv`: Training data (features + labels)
- `datasetTest.csv`: Test data (μόνο features)

**Output:**
- `X`: Training features (N × D)
- `y`: Training labels (N,)
- `X_test`: Test features (M × D)

In [ ]:
def load_data(train_path='Datasets/datasetTV.csv', test_path='Datasets/datasetTest.csv'):
    """Φορτώνει τα δεδομένα εκπαίδευσης και test."""
    paths_train = [train_path, f'../{train_path}', 'train.csv', '../train.csv']
    paths_test = [test_path, f'../{test_path}', 'test.csv', '../test.csv']
    
    for p in paths_train:
        if os.path.exists(p):
            train_path = p
            break
    
    for p in paths_test:
        if os.path.exists(p):
            test_path = p
            break
    
    print(f"Loading: {train_path}, {test_path}")
    
    train_df = pd.read_csv(train_path, header=None)
    X = train_df.iloc[:, :-1].values.astype(np.float32)
    y = train_df.iloc[:, -1].values
    
    if os.path.exists(test_path):
        test_df = pd.read_csv(test_path, header=None)
        X_test = test_df.values.astype(np.float32)
    else:
        X_test = None
    
    return X, y, X_test

# Test loading
X, y, X_test = load_data()
print(f"Train: {X.shape}, Test: {X_test.shape if X_test is not None else 'N/A'}")
print(f"Classes: {np.unique(y)}")

## 4. Pseudo-Labels & Voting

**Σκοπός:** Υποδομή για Self-Training με pseudo-labels.

**Μηχανισμός:**
1. Εκπαίδευση αρχικών μοντέλων
2. Πρόβλεψη στο test set
3. Επιλογή samples με:
   - Confidence ≥ 95%
   - 100% agreement μεταξύ seeds
   - 50%+ agreement μεταξύ views
4. Επανεκπαίδευση με pseudo-labels

**Soft vs Hard Labels:** Χρησιμοποιούμε soft labels (probability vectors) για καλύτερη διατήρηση uncertainty.

In [ ]:
@dataclass(frozen=True)
class PseudoData:
    idx: np.ndarray
    y: np.ndarray
    w: np.ndarray

    @staticmethod
    def empty():
        return PseudoData(
            idx=np.array([], dtype=np.int64),
            y=np.array([], dtype=np.int64),
            w=np.array([], dtype=np.float32),
        )

    def active(self):
        return self.idx is not None and self.y is not None and len(self.idx) > 0

    def is_soft(self):
        return self.y.ndim > 1 or np.issubdtype(self.y.dtype, np.floating)


def normalize_pseudo(pseudo_idx=None, pseudo_y=None, pseudo_w=None):
    if pseudo_idx is None or pseudo_y is None:
        return PseudoData.empty()
    idx = np.asarray(pseudo_idx, dtype=np.int64)
    y = np.asarray(pseudo_y)
    if y.ndim == 1:
        y = y.astype(np.int64)
    else:
        y = y.astype(np.float32)
    w = np.ones((len(idx),), dtype=np.float32) if pseudo_w is None else np.asarray(pseudo_w, dtype=np.float32)
    if len(idx) == 0:
        return PseudoData.empty()
    return PseudoData(idx=idx, y=y, w=w)


def vote_mode_and_agreement(votes_2d):
    mode_pred = np.zeros((votes_2d.shape[1],), dtype=np.int64)
    agree_frac = np.zeros((votes_2d.shape[1],), dtype=np.float64)
    for j in range(votes_2d.shape[1]):
        vals, counts = np.unique(votes_2d[:, j], return_counts=True)
        k = int(np.argmax(counts))
        mode_pred[j] = int(vals[k])
        agree_frac[j] = float(np.max(counts)) / float(votes_2d.shape[0])
    return mode_pred, agree_frac


def view_agreement_fraction(preds_tensor_vs_n, mode_pred):
    view_agree_frac = np.zeros((preds_tensor_vs_n.shape[2],), dtype=np.float64)
    for vi in range(preds_tensor_vs_n.shape[0]):
        view_votes = preds_tensor_vs_n[vi]
        view_mode, _ = vote_mode_and_agreement(view_votes)
        view_agree_frac += (view_mode == mode_pred).astype(np.float64)
    view_agree_frac /= float(preds_tensor_vs_n.shape[0])
    return view_agree_frac

## 5. Συναρτήσεις Απώλειας και Βοηθητικά Εργαλεία

**Σκοπός:** Η χρήση εξειδικευμένων συναρτήσεων απώλειας (loss functions) και εργαλείων για την αντιμετώπιση προβλημάτων όπως το Class Imbalance, τα Noisy Labels και η ανάγκη για Soft Targets.

**Υλοποιήσεις:**
1. **`soft_target_ce` (Soft Cross-Entropy):**
   - Επέκταση της τυπικής Cross-Entropy ώστε να δέχεται πιθανότητες (soft targets) αντί για σκληρές ετικέτες (0/1).
   - Απαραίτητη για **Label Smoothing**, **MixUp** και **Self-Training** (όπου τα pseudo-labels είναι πιθανότητες).

2. **`soft_target_focal` (Focal Loss):**
   - Προσαρμογή του Focal Loss για soft targets.
   - Μειώνει το βάρος στα "εύκολα" παραδείγματα και εστιάζει στα "δύσκολα" (misclassified).
   - Ιδανικό για πολύ ανισοκατανεμημένα δεδομένα (imbalanced datasets).

3. **`compute_class_balanced_weights`:**
   - Υπολογισμός βαρών κλάσεων βασισμένος στον "ενεργό αριθμό δειγμάτων" (Effective Number of Samples) και όχι απλά στη συχνότητα.
   - Τύπος: $W_c = \frac{1 - \beta}{1 - \beta^{N_c}}$ (Cui et al., CVPR 2019).

4. **`prob_meta_features`:**
   - Εξάγει στατιστικά χαρακτηριστικά από τις προβλέψεις πιθανοτήτων (logits/probs).
   - Περιλαμβάνει: Max Confidence, Margin (Gap between 1st & 2nd), Entropy.
   - Χρησιμεύει ως input για τον Stacking Meta-Learner.

In [ ]:
def compute_class_balanced_weights(y, num_classes, beta=0.999):
    """
    Υπολογίζει βάρη κλάσεων βάσει του "Effective Number of Samples".
    Πιο robust από το απλό inverse frequency.
    Paper: Class-Balanced Loss Based on Effective Number of Samples (CVPR 2019)
    """
    counts = np.bincount(np.asarray(y, dtype=np.int64), minlength=num_classes).astype(np.float64)
    effective = 1.0 - np.power(beta, counts)
    weights = (1.0 - beta) / (effective + 1e-12)
    weights = weights / (weights.mean() + 1e-12)
    return weights.astype(np.float32)


def smooth_targets(targets, smoothing):
    """
    Εφαρμόζει Label Smoothing στα targets.
    Αντί για 0 και 1, τα targets γίνονται e/K και 1 - e + e/K.
    Αποτρέπει το μοντέλο από το να γίνεται υπερβολικά σίγουρο (overconfidence).
    """
    if smoothing <= 0:
        return targets
    n_classes = targets.shape[1]
    return targets * (1.0 - smoothing) + (smoothing / n_classes)


def soft_target_ce(logits, targets, class_weights=None):
    """
    Υπολογίζει Cross Entropy Loss με soft targets (πιθανότητες).
    Χρήσιμο για Self-Training (pseudo-labels) και MixUp.
    
    Args:
        pred: Logits από το μοντέλο
        targets: Soft labels (πιθανότητες) ή one-hot encoded targets
        class_weights: Προαιρετικά βάρη για κάθε κλάση (για class imbalance)
    """
    log_probs = F.log_softmax(logits, dim=1)
    if class_weights is not None:
        w = class_weights.view(1, -1)
        return -(targets * w * log_probs).sum(dim=1).mean()
    return -(targets * log_probs).sum(dim=1).mean()


def soft_target_focal(logits, targets, gamma=2.0, class_weights=None):
    """
    Υπολογίζει Focal Loss με soft targets.
    Το Focal Loss εστιάζει περισσότερο στα "δύσκολα" παραδείγματα μειώνοντας
    τη συνεισφορά των εύκολων (well-classified) δειγμάτων.
    
    Args:
        pred: Logits από το μοντέλο
        targets: Soft labels
        gamma: Focusing parameter (συνήθως 2.0). Μεγαλύτερο gamma -> μεγαλύτερη εστίαση στα δύσκολα.
    """
    probs = torch.softmax(logits, dim=1).clamp(1e-8, 1.0 - 1e-8)
    logp = torch.log(probs)
    mod = torch.pow(1.0 - probs, gamma)
    if class_weights is not None:
        w = class_weights.view(1, -1)
        loss = -(targets * w * mod * logp).sum(dim=1)
    else:
        loss = -(targets * mod * logp).sum(dim=1)
    return loss.mean()


def prob_meta_features(probs, lid=None):
    p = np.asarray(probs, dtype=np.float64)
    p = np.clip(p, 1e-12, 1.0)
    p = p / (p.sum(axis=1, keepdims=True) + 1e-12)
    part = np.partition(p, kth=(-1, -2), axis=1)
    top1, top2 = part[:, -1], part[:, -2]
    gap = top1 - top2
    entropy = -(p * np.log(p)).sum(axis=1)
    feats = np.column_stack([top1, gap, entropy])
    if lid is not None:
        feats = np.column_stack([feats, np.asarray(lid, dtype=np.float64).reshape(-1)])
    return feats.astype(np.float32)

## 6. Domain Adaptation

**Σκοπός:** Αντιμετώπιση του **distribution shift (covariate shift)**, δηλαδή της διαφοράς στην κατανομή των δεδομένων μεταξύ Training και Test set. Αυτό είναι κρίσιμο σε transductive settings όπου έχουμε πρόσβαση στα unlabeled test δεδομένα.

**Τεχνικές που υλοποιούνται:**
1. **Adversarial Reweighting (Importance Sampling):**
   - Εκπαιδεύουμε έναν classifier (Domain Discriminator) να ξεχωρίζει τα Train από τα Test δείγματα.
   - Υπολογίζουμε βάρη σημαντικότητας $w(x) \approx \frac{p_{test}(x)}{p_{train}(x)}$.
   - Δίνουμε μεγαλύτερο βάρος στα training samples που μοιάζουν με test samples.
   - *Υλοποίηση*: Συνάρτηση `adversarial_weights`.

2. **CORAL (Correlation Alignment):**
   - Ευθυγραμμίζει τους πίνακες συνδιακύμανσης (covariance matrices) των χαρακτηριστικών μεταξύ Train και Test.
   - Ελαχιστοποιεί την απόσταση $||C_{train} - C_{test}||_F^2$.
   - Μετασχηματίζει τα features ώστε να έχουν παρόμοια στατιστική δομή.

**Πότε χρησιμοποιείται:**
- Όταν παρατηρείται διαφορά στην απόδοση μεταξύ CV και Leaderboard.
- Όταν τα features έχουν διαφορετικές κατανομές (π.χ. λόγω χρονικής μετατόπισης).
- Ενεργοποιείται μέσω `ENABLE_ADV_REWEIGHT` και `ENABLE_CORAL`.

In [ ]:
def adversarial_weights(X_train, X_test, seed=42, model='lr', clip=10.0, power=1.0):
    X_all = np.vstack([X_train, X_test])
    y_dom = np.concatenate([np.zeros(len(X_train), dtype=np.int64), np.ones(len(X_test), dtype=np.int64)])
    clf = LogisticRegression(max_iter=2000)
    clf.fit(X_all, y_dom)
    p_test = clf.predict_proba(X_train)[:, 1].astype(np.float64)
    p_test = np.clip(p_test, 1e-6, 1.0 - 1e-6)
    w = p_test / (1.0 - p_test)
    w = np.power(w, float(power))
    w = np.clip(w, 1.0 / float(clip), float(clip))
    w = w / (np.mean(w) + 1e-12)
    return w.astype(np.float32)


def coral_align(X_train, X_test, reg=1e-3):
    X_tr = np.asarray(X_train, dtype=np.float64)
    X_te = np.asarray(X_test, dtype=np.float64)
    X_trc = X_tr - X_tr.mean(axis=0, keepdims=True)
    X_tec = X_te - X_te.mean(axis=0, keepdims=True)
    cov_tr = (X_trc.T @ X_trc) / max(1, (len(X_trc) - 1)) + reg * np.eye(X_tr.shape[1])
    cov_te = (X_tec.T @ X_tec) / max(1, (len(X_tec) - 1)) + reg * np.eye(X_te.shape[1])
    evals_tr, evecs_tr = np.linalg.eigh(cov_tr)
    evals_tr = np.clip(evals_tr, 1e-12, None)
    W_tr = evecs_tr @ np.diag(1.0 / np.sqrt(evals_tr)) @ evecs_tr.T
    evals_te, evecs_te = np.linalg.eigh(cov_te)
    evals_te = np.clip(evals_te, 1e-12, None)
    C_te = evecs_te @ np.diag(np.sqrt(evals_te)) @ evecs_te.T
    A = W_tr @ C_te
    X_tr_a = X_trc @ A + X_tr.mean(axis=0, keepdims=True)
    return X_tr_a.astype(np.float32), X_te.astype(np.float32)

## 7. Feature Engineering (Transductive & Manifold)

**Σκοπός:** Εμπλουτισμός των δεδομένων με βαθιές αναπαραστάσεις που εκμεταλλεύονται τη γεωμετρία (manifold) και τη μη-επιβλεπόμενη μάθηση (unsupervised learning) στο σύνολο των δεδομένων (train + test).

**Τεχνικές Eπαύξησης (Augmentation):**
1. **Transductive DAE (Denoising Autoencoder):**
   - Neural network (swap noise) που μαθαίνει να ανακατασκευάζει τα δεδομένα.
   - Εκπαιδεύεται σε Train + Test (χωρίς labels), μαθαίνοντας την υποκείμενη δομή του χώρου.
   - Το hidden layer (embedding) χρησιμοποιείται ως νέα features.
   - *Οφέλη:* Robustness σε θόρυβο, πυκνή αναπαράσταση.

2. **Manifold Features (Graph-based):**
   - Κατασκευή k-NN γράφου που συνδέει δείγματα με βάση την ομοιότητα.
   - **LID (Local Intrinsic Dimensionality):** Μετρά την πολυπλοκότητα της γειτονιάς ενός σημείου. Υψηλό LID → δύσκολο σημείο/outlier.
   - **PageRank:** Μετρά την "κεντρικότητα" (centrality) κάθε δείγματος στον γράφο. Δείκτης πυκνότητας.
   - **Laplacian Eigenmaps (Spectral Embedding):** Μη-γραμμική μείωση διαστάσεων που διατηρεί την τοπική δομή του manifold.
     - *Υλοποίηση:* GPU-accelerated με `torch.lobpcg` (PyTorch) ή CPU fallback (`sklearn`).

**Data Views (Προβολές Δεδομένων):**
- `raw`: Αρχικά χαρακτηριστικά (κατάλληλα για GBDTs και TabPFN).
- `quantile`: Rank-based normalization (GaussRank). Κάνει τα features να ακολουθούν κανονική κατανομή. Απαραίτητο για Neural Networks (TabR).
- `pca`/`ica`: Γραμμική μείωση διαστάσεων για αποθορυβοποίηση.

In [ ]:
def gpu_laplacian_eigenmaps(X, n_components=8, k=20, device=None):
    """GPU-accelerated Laplacian Eigenmaps using torch.lobpcg."""
    if device is None:
        device = config.DEVICE
    n = X.shape[0]
    k_eff = min(k, n - 1)
    A_sparse = kneighbors_graph(X, k_eff, mode='connectivity', include_self=False)
    A_sparse = (A_sparse + A_sparse.T) / 2
    A_sparse = A_sparse.tocoo()
    indices = torch.tensor(np.vstack([A_sparse.row, A_sparse.col]), dtype=torch.long, device=device)
    values = torch.tensor(A_sparse.data, dtype=torch.float32, device=device)
    A = torch.sparse_coo_tensor(indices, values, (n, n)).coalesce()
    deg = torch.sparse.sum(A, dim=1).to_dense()
    deg_inv_sqrt = torch.where(deg > 0, deg.pow(-0.5), torch.zeros_like(deg))
    row, col = A.indices()
    scaled_vals = A.values() * deg_inv_sqrt[row] * deg_inv_sqrt[col]
    A_norm = torch.sparse_coo_tensor(A.indices(), scaled_vals, (n, n)).coalesce()
    n_req = min(n_components + 1, n - 1)
    I_indices = torch.arange(n, device=device).unsqueeze(0).repeat(2, 1)
    I_values = torch.ones(n, device=device, dtype=torch.float32)
    I_sparse = torch.sparse_coo_tensor(I_indices, I_values, (n, n)).coalesce()
    L = (I_sparse - A_norm).coalesce()
    eigenvalues, eigenvectors = torch.lobpcg(L.to_dense(), k=n_req, largest=False, niter=100)
    embedding = eigenvectors[:, 1:n_components + 1].cpu().numpy()
    if embedding.shape[1] < n_components:
        padding = np.zeros((n, n_components - embedding.shape[1]))
        embedding = np.hstack([embedding, padding])
    return embedding


class TransductiveDAE(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(input_dim, 512), nn.SiLU(), nn.Linear(512, 128))
        self.decoder = nn.Sequential(nn.Linear(128, 512), nn.SiLU(), nn.Linear(512, input_dim))

    def forward(self, x):
        return self.decoder(self.encoder(x))


class DataRefinery:
    def __init__(self, input_dim):
        self.dae = TransductiveDAE(input_dim).to(config.DEVICE)

    def fit(self, X_all, epochs=None, noise_std=None):
        if epochs is None: epochs = config.DAE_EPOCHS
        if noise_std is None: noise_std = config.DAE_NOISE_STD
        X_t = torch.tensor(X_all, dtype=torch.float32).to(config.DEVICE)
        dl = DataLoader(TensorDataset(X_t), batch_size=config.BATCH_SIZE, shuffle=True)
        opt = optim.AdamW(self.dae.parameters(), lr=config.LR_SCALE)
        crit = nn.MSELoss()
        self.dae.train()
        for _ in range(int(epochs)):
            for (xb,) in dl:
                noise = torch.randn_like(xb) * float(noise_std)
                rec = self.dae(xb + noise)
                loss = crit(rec, xb)
                opt.zero_grad(); loss.backward(); opt.step()
        return self

    def transform(self, X):
        self.dae.eval()
        X_t = torch.tensor(X, dtype=torch.float32).to(config.DEVICE)
        emb, rec = [], []
        for i in range(0, len(X), config.BATCH_SIZE):
            xb = X_t[i:i+config.BATCH_SIZE]
            with torch.no_grad():
                z = self.dae.encoder(xb)
                r = self.dae.decoder(z)
            emb.append(z.cpu().numpy())
            rec.append(r.cpu().numpy())
        return np.vstack(emb), np.vstack(rec)


def apply_feature_view(X_train, X_test, view, seed, allow_transductive=False):
    view = (view or 'raw').strip().lower()
    if view == 'raw':
        return X_train.copy(), X_test.copy()
    if view == 'quantile':
        qt = QuantileTransformer(output_distribution='normal', random_state=seed)
        return qt.fit_transform(X_train), qt.transform(X_test)
    if view.startswith('pca'):
        n_components = min(50, X_train.shape[1], max(2, X_train.shape[0] - 1))
        pca = PCA(n_components=n_components, random_state=seed)
        return pca.fit_transform(X_train), pca.transform(X_test)
    if view.startswith('ica'):
        n_components = min(50, X_train.shape[1], max(2, X_train.shape[0] - 1))
        ica = FastICA(n_components=n_components, random_state=seed, max_iter=500)
        return ica.fit_transform(X_train), ica.transform(X_test)
    raise ValueError(f"Unknown view: {view}")


def compute_manifold_features(X_train, X_test, allow_transductive=True, k=20, return_lid=False):
    if allow_transductive:
        X_all = np.vstack([X_train, X_test])
        nbrs = NearestNeighbors(n_neighbors=k, n_jobs=-1).fit(X_all)
        dists, _ = nbrs.kneighbors(X_all)
        d_k = dists[:, -1]
        d_j = dists[:, 1:]
        lid = k / np.sum(np.log(d_k[:, None] / (d_j + 1e-10) + 1e-10), axis=1)
        lid = (lid - lid.min()) / (lid.max() - lid.min() + 1e-12)
        
        # PageRank
        try:
            import networkx as nx
            A = kneighbors_graph(X_all, k, mode='connectivity', include_self=False)
            G = nx.from_scipy_sparse_array(A)
            pr = nx.pagerank(G, alpha=0.85, max_iter=50)
            pagerank = np.array([pr[i] for i in range(len(X_all))], dtype=np.float64)
            pagerank = (pagerank - pagerank.min()) / (pagerank.max() - pagerank.min() + 1e-12)
        except:
            pagerank = np.zeros(len(X_all))
        
        # Laplacian
        if config.ENABLE_LAPLACIAN:
            try:
                n_comp = min(8, len(X_all) - 2)
                se = SpectralEmbedding(n_components=n_comp, n_neighbors=k, n_jobs=-1, random_state=42)
                laplacian = se.fit_transform(X_all)
                print(f"   [MANIFOLD] Laplacian Eigenmaps: {n_comp} components")
            except:
                laplacian = np.zeros((len(X_all), 8))
        else:
            laplacian = np.zeros((len(X_all), 0))
        
        if laplacian.shape[1] > 0:
            feats = np.column_stack([lid, pagerank, laplacian])
        else:
            feats = np.column_stack([lid, pagerank])
        
        feats_tr, feats_te = feats[:len(X_train)], feats[len(X_train):]
        if return_lid:
            return feats_tr, feats_te, lid[:len(X_train)], lid[len(X_train):]
        return feats_tr, feats_te
    return np.zeros((len(X_train), 2)), np.zeros((len(X_test), 2))


def build_streams(X_v, X_test_v):
    ref_fit_X = np.vstack([X_v, X_test_v]) if config.ALLOW_TRANSDUCTIVE else X_v
    ref = DataRefinery(X_v.shape[1]).fit(ref_fit_X)
    feats_tr, feats_te, lid_tr, lid_te = compute_manifold_features(
        X_v, X_test_v, allow_transductive=config.ALLOW_TRANSDUCTIVE,
        k=config.MANIFOLD_K, return_lid=True
    )
    emb_tr, rec_tr = ref.transform(X_v)
    emb_te, rec_te = ref.transform(X_test_v)
    X_neural_tr = np.hstack([X_v, feats_tr, emb_tr])
    X_neural_te = np.hstack([X_test_v, feats_te, emb_te])
    X_tree_tr = np.hstack([X_v, feats_tr, rec_tr])
    X_tree_te = np.hstack([X_test_v, feats_te, rec_te])
    return X_tree_tr, X_tree_te, X_neural_tr, X_neural_te, lid_tr, lid_te

## 8. Tree Models (XGBoost, CatBoost)

**Σκοπός:** Gradient Boosting μοντέλα για tabular data.

**XGBoost DART:**
- Dropout Additive Regression Trees
- Αποφυγή overfitting μέσω dropout
- GPU acceleration

**CatBoost Langevin:**
- Stochastic Gradient Langevin Dynamics
- Καλύτερη γενίκευση
- Native GPU support

In [ ]:
def get_xgb_dart(n_c, iterations=500):
    from xgboost import XGBClassifier
    use_gpu = torch.cuda.is_available()
    params = dict(
        booster='dart', rate_drop=0.1, skip_drop=0.5,
        n_estimators=iterations, max_depth=8, learning_rate=0.066,
        subsample=0.96, colsample_bytree=0.62, gamma=0.016,
        reg_alpha=0.0009, reg_lambda=1.14,
        objective='multi:softprob', num_class=int(n_c),
        eval_metric='mlogloss', verbosity=0,
    )
    if use_gpu:
        params.update(tree_method='hist', device='cuda')
    else:
        params.update(tree_method='hist', device='cpu')
    return XGBClassifier(**params)


def get_cat_langevin(n_c, iterations=1000):
    from catboost import CatBoostClassifier
    return CatBoostClassifier(
        langevin=True, diffusion_temperature=1000,
        iterations=iterations, depth=9, learning_rate=0.0485,
        l2_leaf_reg=2.68, random_strength=2.42, bagging_temperature=0.035,
        loss_function='MultiClass', eval_metric='MultiClass',
        task_type='GPU' if torch.cuda.is_available() else 'CPU',
        verbose=0, allow_writing_files=False,
    )

## 9. SAM Optimizer (Sharpness-Aware Minimization)

**Σκοπός:** Η εύρεση παραμέτρων που βρίσκονται σε **"επίπεδα" (flat) ελάχιστα** της συνάρτησης απώλειας, αντί για αιχμηρά (sharp). Τα επίπεδα ελάχιστα γενικεύουν καλύτερα στο Test Set, καθώς μικρές διαταραχές στα δεδομένα δεν αυξάνουν δραματικά το loss.

**Μηχανισμός (Two-Step Optimization):**
Για κάθε βήμα εκπαίδευσης:
1. **Adversarial Step (Ascent):** Υπολογίζει την κατεύθυνση μέγιστης αύξησης του loss (gradient ascent) σε μια μικρή περιοχή $\rho$ γύρω από τα τρέχοντα weights $w$. Βρίσκει δηλαδή το "χειρότερο" σημείο στη γειτονιά.
   $$ \epsilon = \rho \frac{\nabla L(w)}{||\nabla L(w)||} $$
2. **Optimization Step (Descent):** Υπολογίζει το gradient σε αυτό το "χειρότερο" σημείο $w+\epsilon$ και κάνει το update στα αρχικά βάρη $w$.

**Πλεονεκτήματα:**
- **Γενίκευση:** Μειώνει δραστικά το χάσμα μεταξύ Train και Test accuracy.
- **Robustness:** Το μοντέλο γίνεται πιο ανθεκτικό σε θόρυβο.
- *Κόστος:* Διπλασιάζει τον χρόνο εκπαίδευσης (2 forward/backward per step), αλλά αξίζει για το performance boost.

In [ ]:
class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, adaptive=False, **kwargs):
        defaults = dict(rho=rho, adaptive=adaptive, **kwargs)
        super().__init__(params, defaults)
        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups
        self.defaults.update(self.base_optimizer.defaults)

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group['rho'] / (grad_norm + 1e-12)
            for p in group['params']:
                if p.grad is None: continue
                self.state[p]['old_p'] = p.data.clone()
                p.add_((torch.pow(p, 2) if group['adaptive'] else 1.0) * p.grad * scale.to(p))
        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None: continue
                p.data = self.state[p]['old_p']
        if zero_grad: self.zero_grad()

    def _grad_norm(self):
        shared_device = self.param_groups[0]['params'][0].device
        return torch.norm(torch.stack([
            ((torch.abs(p) if group['adaptive'] else 1.0) * p.grad).norm(p=2).to(shared_device)
            for group in self.param_groups for p in group['params'] if p.grad is not None
        ]), p=2)

    def step(self): raise NotImplementedError

## 10. TrueTabR (Retrieval-Augmented Neural Network)

**Σκοπός:** Ένα καινοτόμο Neural Network ειδικά σχεδιασμένο για Tabular Data, το οποίο ενσωματώνει μηχανισμό ανάκτησης (retrieval mechanism) παρόμοιο με τα Large Language Models (RAG), επιτρέποντας στο δίκτυο να κοιτάζει "παρόμοια παραδείγματα" κατά την πρόβλεψη.

**Αρχιτεκτονική (Attention-Based):**
1. **Encoder:** Το input δείγμα $x$ περνάει από MLP και γίνεται query embedding $q$.
2. **Retrieval:** Βρίσκονται οι $k$ πλησιέστεροι γείτονες (Neighbors $N$) του $x$ από το Training Set (χρήση `NearestNeighbors` με `scikit-learn`).
3. **Attention Mechanism:**
   - Οι neighbors κωδικοποιούνται σε κλειδιά(keys $k$) και τιμές(values $v$).
   - Υπολογίζεται το `context` μέσω Scaled Dot-Product Attention: 
     $$ \text{context} = \text{softmax}\left(\frac{q \cdot k^T}{\sqrt{d}}\right) \cdot v $$
4. **Fusion & Output:** Το `context` (πληροφορία από γείτονες) προστίθεται στο αρχικό query $q$ (residual connection) και περνάει από τελικό MLP για να παραχθούν τα class logits.

**Πλεονεκτήματα:**
- **Context-Aware:** Δεν βασίζεται μόνο στα weights, αλλά και στα πραγματικά δεδομένα κατά το inference.
- **Robustness:** Εξαιρετικά ανθεκτικό σε rare patterns/outliers, καθώς μπορεί να ανασύρει σχετικά παραδείγματα.
- **State-of-the-Art:** Συχνά ξεπερνά τα GBDTs σε πολύπλοκα tabular datasets.

In [ ]:
class TabRModule(nn.Module):
    def __init__(self, input_dim, num_classes, context_size=96, hidden_dim=128, head_hidden=64):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(input_dim, hidden_dim), nn.SiLU(), nn.Linear(hidden_dim, context_size))
        self.q_proj = nn.Linear(context_size, context_size)
        self.k_proj = nn.Linear(context_size, context_size)
        self.v_proj = nn.Linear(context_size, context_size)
        self.head = nn.Sequential(nn.Linear(context_size, head_hidden), nn.SiLU(), nn.Linear(head_hidden, num_classes))
        self.scale = context_size ** -0.5

    def forward(self, x, neighbors):
        q = self.encoder(x).unsqueeze(1)
        B, K, D = neighbors.shape
        kv = self.encoder(neighbors.view(B * K, D)).view(B, K, -1)
        scores = torch.bmm(self.q_proj(q), self.k_proj(kv).transpose(1, 2)) * self.scale
        context = torch.bmm(F.softmax(scores, dim=-1), self.v_proj(kv)).squeeze(1)
        return self.head(context + q.squeeze(1))


class TrueTabR(BaseEstimator, ClassifierMixin):
    def __init__(self, num_classes, n_neighbors=16, context_size=96, hidden_dim=128, head_hidden=64):
        self.num_classes, self.n_neighbors = num_classes, n_neighbors
        self.context_size, self.hidden_dim, self.head_hidden = context_size, hidden_dim, head_hidden
        self.model, self.knn, self.X_train_ = None, None, None

    def fit(self, X, y, sample_weight=None, epochs=20):
        self.X_train_ = np.array(X, dtype=np.float32)
        self.knn = NearestNeighbors(n_neighbors=self.n_neighbors, n_jobs=-1).fit(self.X_train_)
        train_neighbor_idx = self.knn.kneighbors(self.X_train_, return_distance=False)
        self.model = TabRModule(X.shape[1], self.num_classes, self.context_size, self.hidden_dim, self.head_hidden).to(config.DEVICE)
        opt = optim.AdamW(self.model.parameters(), lr=config.LR_SCALE)
        
        class_w = None
        if config.USE_CLASS_BALANCED:
            class_w_np = compute_class_balanced_weights(y, self.num_classes, beta=config.CB_BETA)
            class_w = torch.tensor(class_w_np, dtype=torch.float32, device=config.DEVICE)

        crit = nn.CrossEntropyLoss(weight=class_w, label_smoothing=float(config.LABEL_SMOOTHING) if config.LABEL_SMOOTHING > 0 else 0.0)

        X_t = torch.tensor(X, dtype=torch.float32)
        y_t = torch.tensor(y, dtype=torch.long)
        idx_t = torch.arange(len(X_t), dtype=torch.long)
        dl = DataLoader(TensorDataset(X_t, y_t, idx_t), batch_size=config.BATCH_SIZE, shuffle=True)

        self.model.train()
        for _ in range(epochs):
            for xb, yb, ib in dl:
                nx = self.X_train_[train_neighbor_idx[ib.numpy()]]
                logits = self.model(xb.to(config.DEVICE), torch.tensor(nx, dtype=torch.float32).to(config.DEVICE))
                loss = crit(logits, yb.to(config.DEVICE))
                opt.zero_grad(); loss.backward(); opt.step()
        return self

    def predict_proba(self, X):
        self.model.eval()
        p = []
        for i in range(0, len(X), config.BATCH_SIZE):
            xb = X[i:i + config.BATCH_SIZE]
            nx = self.X_train_[self.knn.kneighbors(xb, return_distance=False)]
            with torch.no_grad():
                p.append(torch.softmax(
                    self.model(torch.tensor(xb, dtype=torch.float32).to(config.DEVICE), torch.tensor(nx).to(config.DEVICE)),
                    dim=1).cpu().numpy())
        return np.vstack(p)
    
    def finetune_on_pseudo(self, X_pseudo, y_pseudo, epochs=1, lr_mult=0.2):
        # Simplified - full implementation in original code
        return self

## 11. TabPFN Wrapper (Post-Hoc Ensembling)

**Σκοπός:** Foundation Model για tabular classification.

**TabPFN v2.5:**
- Pre-trained transformer για in-context learning
- Δεν απαιτεί training, μόνο inference
- Υποστηρίζει έως 50K samples

**Post-Hoc Ensembling (PHE):**
- Αυτόματη επιλογή 100 base models
- Greedy ensemble selection
- Optimal weight optimization

In [ ]:
class TabPFNWrapper(BaseEstimator, ClassifierMixin):
    """TabPFN v2.5 with AutoTabPFNClassifier (PHE)."""
    def __init__(self, device='cuda', n_estimators=32, random_state=42):
        self.device = device
        self.n_estimators = n_estimators
        self.random_state = random_state
        self.model_ = None
        self.classes_ = None

    def fit(self, X, y, sample_weight=None):
        try:
            from tabpfn_extensions.post_hoc_ensembles.sklearn_interface import AutoTabPFNClassifier
        except ImportError:
            print("[TabPFN] WARNING: tabpfn-extensions not installed. Using random fallback.")
            self.model_ = None
            self.classes_ = np.unique(y)
            return self

        self.classes_ = np.unique(y)
        X_np = np.asarray(X)
        y_np = np.asarray(y)

        self.model_ = AutoTabPFNClassifier(
            device=self.device,
            random_state=self.random_state,
            max_time=config.TABPFN_MAX_TIME,
            ignore_pretraining_limits=True
        )
        self.model_.fit(X_np, y_np)
        print(f"[TabPFN Auto] Fitted with {X_np.shape[0]} samples, {X_np.shape[1]} features, on {self.device} (PHE enabled)")
        return self

    def predict_proba(self, X):
        if self.model_ is None:
            return np.ones((len(X), len(self.classes_))) / len(self.classes_)
        return self.model_.predict_proba(np.asarray(X))

    def predict(self, X):
        probs = self.predict_proba(X)
        return self.classes_[np.argmax(probs, axis=1)]

## 12. Calibration (Βαθμονόμηση Πιθανοτήτων)

**Σκοπός:** Η ευθυγράμμιση των προβλεπόμενων πιθανοτήτων (predicted probabilities) με τις πραγματικές συχνότητες εμφάνισης των κλάσεων. Πολλά σύγχρονα μοντέλα (π.χ. Neural Networks, Random Forests) τείνουν να είναι "overconfident" ή "underconfident".

**Διαδικασία (CalibratedModel class):**
1. **K-Fold CV Training:** Το Training set χωρίζεται σε K folds (αυστηρά Stratified). Το μοντέλο εκπαιδεύεται K φορές σε (K-1) folds και κάνει προβλέψεις στο αθέατο validation fold.
2. **Isotonic Regression:** Για κάθε κλάση, εκπαιδεύεται ένας Isotonic Regressor (μη-παραμετρική μονότονη συνάρτηση) που map-άρει τα raw outputs (logits/probs) σε βαθμονομημένες πιθανότητες, χρησιμοποιώντας τις validation προβλέψεις.
3. **Inference (Test time):** Κατά την πρόβλεψη, κάθε ένα από τα K μοντέλα παράγει μια πρόβλεψη, η οποία βαθμονομείται από τον αντίστοιχο Regressor. Το τελικό αποτέλεσμα είναι ο μέσος όρος των K βαθμονομημένων προβλέψεων.

**Αποτέλεσμα:**
- Πιο αξιόπιστες πιθανότητες, κρίσιμες για το **Stacking** (που χρησιμοποιεί τις πιθανότητες ως input) και το **Self-Training** (που βασίζεται στο confidence threshold).
- Μείωση του log-loss.
- Εξομάλυνση υπερβολικής βεβαιότητας (overconfidence mitigation).

In [ ]:
from sklearn.isotonic import IsotonicRegression

class CalibratedModel:
    def __init__(self, base_model, name):
        self.base, self.name = base_model, name
        self.models = []
        self.calibrators = []

    def fit(self, X, y, sample_weight=None, pseudo_X=None, pseudo_y=None, pseudo_w=None):
        skf = StratifiedKFold(n_splits=config.N_FOLDS, shuffle=True, random_state=42)
        self.models = []
        self.calibrators = []
        self.cv_scores = []

        for fold_i, (tr_idx, val_idx) in enumerate(skf.split(X, y)):
            X_tr, X_val = X[tr_idx], X[val_idx]
            y_tr, y_val = y[tr_idx], y[val_idx]

            model = copy.deepcopy(self.base)
            try:
                model.fit(X_tr, y_tr, sample_weight=sample_weight[tr_idx] if sample_weight is not None else None)
            except TypeError:
                model.fit(X_tr, y_tr)

            val_probs = model.predict_proba(X_val).astype(np.float32)
            val_preds = np.argmax(val_probs, axis=1)
            fold_acc = (val_preds == y_val).mean()
            self.cv_scores.append(fold_acc)

            c_list = []
            for c in range(val_probs.shape[1]):
                iso = IsotonicRegression(out_of_bounds='clip')
                iso.fit(val_probs[:, c], (y_val == c).astype(int))
                c_list.append(iso)

            self.models.append(model)
            self.calibrators.append(c_list)

        mean_cv = np.mean(self.cv_scores)
        print(f"      CV Score: {mean_cv:.2%}")
        return self

    def predict_proba(self, X):
        total_probs = np.zeros((len(X), len(self.calibrators[0])))
        for model, calib_list in zip(self.models, self.calibrators):
            raw_p = model.predict_proba(X)
            cal_p = np.zeros_like(raw_p)
            for c in range(raw_p.shape[1]):
                cal_p[:, c] = calib_list[c].predict(raw_p[:, c])
            cal_p /= (cal_p.sum(axis=1, keepdims=True) + 1e-10)
            total_probs += cal_p
        return total_probs / len(self.models)

## 13. Diffusion Augmentation (SMOTE Evolved)

**Σκοπός:** Η δημιουργία υψηλής ποιότητας συνθετικών δειγμάτων (synthetic samples) για κάθε κλάση, λειτουργώντας ως μια σύγχρονη εναλλακτική του SMOTE για Data Augmentation σε tabular data.

**Μηχανισμός (Sklearn Interface):**
1. **Προσαρμογή:** Εκπαιδεύεις ένα μοντέλο διάχυσης (Denoising Diffusion Probabilistic Model - DDPM) για κάθε κλάση ξεχωριστά, μαθαίνοντας την κατανομή των πραγματικών δεδομένων.
2. **Forward Process:** Προσθήκη Gaussian θορύβου στα πραγματικά δεδομένα μέχρι να γίνουν pure noise.
3. **Reverse Process (Generation):** Το δίκτυο μαθαίνει να αφαιρεί τον θόρυβο βήμα-βήμα, ξεκινώντας από τυχαίο θόρυβο και καταλήγοντας σε νέα, ρεαλιστικά δείγματα της κλάσης.
4. **Επαύξηση:** Τα παραγόμενα δείγματα προστίθενται στο Training Set.

**Πλεονεκτήματα:**
- **High Fidelity:** Παράγει πιο ρεαλιστικά δείγματα από το γραμμικό interpolation του SMOTE.
- **Class Balancing:** Βοηθά στην εξισορρόπηση των κλάσεων αντιγράφοντας τη δομή της μειοψηφούσας κλάσης.
- **Regularization:** Λειτουργεί ως data-driven regularization, αποτρέποντας το overfitting.

*Σημείωση: Απαιτεί σημαντικό χρόνο εκπαίδευσης, γι' αυτό χρησιμοποιείται επιλεκτικά (ENABLE_DIFFUSION).*

In [ ]:
class TabularDiffusion(nn.Module):
    def __init__(self, dim, hidden_dim=512):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim), nn.SiLU(), nn.Dropout(0.1),
            nn.Linear(hidden_dim, hidden_dim), nn.SiLU(), nn.Dropout(0.1),
            nn.Linear(hidden_dim, dim)
        )

    def forward(self, x):
        return self.net(x)


def synthesize_data(X, y, n_new_per_class=200):
    if len(X) < 50:
        return X, y
    X_syn_all, y_syn_all = [], []
    classes = np.unique(y)
    print(f"   [DIFFUSION] Synthesizing {n_new_per_class} samples/class...")
    for c in classes:
        Xc = X[y == c]
        if len(Xc) < 10: continue
        model = TabularDiffusion(Xc.shape[1]).to(config.DEVICE)
        opt = optim.Adam(model.parameters(), lr=1e-3)
        Xt = torch.tensor(Xc, dtype=torch.float32).to(config.DEVICE)
        model.train()
        for _ in range(int(config.DIFFUSION_EPOCHS)):
            noise = torch.randn_like(Xt) * 0.1
            rec = model(Xt + noise)
            loss = F.mse_loss(rec, Xt)
            opt.zero_grad(); loss.backward(); opt.step()
        model.eval()
        with torch.no_grad():
            idx = np.random.choice(len(Xc), n_new_per_class, replace=True)
            seed = Xt[idx] + torch.randn_like(Xt[idx]) * 0.2
            gen = model(seed).cpu().numpy()
            X_syn_all.append(gen)
            y_syn_all.append(np.full(n_new_per_class, c))
    if not X_syn_all:
        return X, y
    return np.vstack([X, np.vstack(X_syn_all)]), np.concatenate([y, np.concatenate(y_syn_all)])

## 14. Stacking Meta-Learner

**Σκοπός:** Συνδυασμός base models μέσω cross-fit stacking.

**Διαδικασία:**
```
Training Data
    ↓
K-Fold Split
    ↓
Train Base Models on K-1 folds
    ↓
Predict on held-out fold (OOF predictions)
    ↓
Stack OOF predictions + Meta-features
    ↓
Train Meta-Learner (LogisticRegression C=0.55)
    ↓
Final predictions on test
```

**Meta-features:** confidence, gap, entropy για κάθε base model.

In [ ]:
def fit_predict_stacking(names_models, view_name, X_train_base, X_test_base, y, num_classes,
                         cv_splits=5, seed=42, sample_weight=None,
                         pseudo_idx=None, pseudo_y=None, pseudo_w=None, return_oof=False):
    skf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=seed)
    n_models = len(names_models)
    model_map = {name: i for i, (name, _) in enumerate(names_models)}
    
    oof_preds = [np.zeros((len(y), num_classes), dtype=np.float32) for _ in range(n_models)]
    test_preds_running = [np.zeros((len(X_test_base), num_classes), dtype=np.float32) for _ in range(n_models)]

    print(f"  [STACKING] Cross-Validation ({cv_splits} folds) | View: {view_name} | Models: {len(names_models)}")
    
    X_tr_view, X_test_view_fold = apply_feature_view(X_train_base, X_test_base, view=view_name, seed=seed, allow_transductive=config.ALLOW_TRANSDUCTIVE)
    
    for tr_idx, val_idx in skf.split(X_tr_view, y):
        X_tr_fold = X_tr_view[tr_idx]
        X_val_fold = X_tr_view[val_idx]
        y_tr = y[tr_idx]

        # Diffusion Augmentation
        if config.ENABLE_DIFFUSION and len(X_tr_fold) > 100:
            X_tr_fold, y_tr = synthesize_data(X_tr_fold, y_tr, n_new_per_class=config.DIFFUSION_N_SAMPLES // 5)
        
        X_tree_tr, X_tree_val, X_neural_tr, X_neural_val, _, _ = build_streams(X_tr_fold, X_val_fold)
        _, X_tree_te_fold, _, X_neural_te_fold, _, _ = build_streams(X_tr_fold, X_test_view_fold)

        for name, base_template in names_models:
            idx_m = model_map[name]
            model = copy.deepcopy(base_template)
            is_tree = ('XGB' in name or 'Cat' in name or 'LGBM' in name)
            X_f_tr = X_tree_tr if is_tree else X_neural_tr
            X_f_val = X_tree_val if is_tree else X_neural_val
            X_f_te = X_tree_te_fold if is_tree else X_neural_te_fold

            try:
                model.fit(X_f_tr, y_tr)
            except TypeError:
                model.fit(X_f_tr, y_tr)
            
            p_oof = model.predict_proba(X_f_val).astype(np.float32)
            oof_preds[idx_m][val_idx] = p_oof
            p_test = model.predict_proba(X_f_te).astype(np.float32)
            test_preds_running[idx_m] += p_test
            
            del model
            import gc; gc.collect()
            if torch.cuda.is_available(): torch.cuda.empty_cache()

    for i in range(n_models):
        test_preds_running[i] /= cv_splits

    # Meta Learner
    meta_X = np.hstack(oof_preds)
    meta_te = np.hstack(test_preds_running)
    
    meta_feat_oof = [prob_meta_features(oof) for oof in oof_preds]
    meta_feat_te = [prob_meta_features(p) for p in test_preds_running]
    meta_X = np.hstack([meta_X] + meta_feat_oof)
    meta_te = np.hstack([meta_te] + meta_feat_te)

    if return_oof:
        return oof_preds, test_preds_running, meta_X, meta_te, y

    print(f"  > Fitting Meta-Learner ({config.META_LEARNER})...")
    
    if config.META_LEARNER == 'lr':
        meta = LogisticRegression(C=0.55, random_state=seed, solver='lbfgs', max_iter=1000)
        meta.fit(meta_X, y)
        final_probs = meta.predict_proba(meta_te)
    elif config.META_LEARNER == 'ridge':
        meta = RidgeClassifier(alpha=1.0, random_state=seed)
        meta.fit(meta_X, y)
        d = meta.decision_function(meta_te)
        e_d = np.exp(d - np.max(d, axis=1, keepdims=True))
        final_probs = e_d / e_d.sum(axis=1, keepdims=True)
    else:  # NNLS
        n_base_models = len(oof_preds)
        Z = np.zeros((len(y), n_base_models))
        for m_i in range(n_base_models):
            Z[:, m_i] = oof_preds[m_i][np.arange(len(y)), y]
        target = np.ones(len(y))
        weights, _ = nnls(Z, target)
        weights /= (weights.sum() + 1e-9)
        print(f"   [NNLS Weights] {weights}")
        final_probs = np.zeros_like(test_preds_running[0])
        for m_i in range(n_base_models):
            final_probs += test_preds_running[m_i] * weights[m_i]
    
    return final_probs

## 15. Per-View Pipeline (Smart Routing)

**Σκοπός:** Η ενορχήστρωση της διαδικασίας πρόβλεψης για κάθε διαφορετική "οπτική" (view) των δεδομένων, διασφαλίζοντας ότι κάθε μοντέλο λαμβάνει την είσοδο που του ταιριάζει καλύτερα.

**Smart View Routing:**

| View | Models | Λογική |
|---|---|---|
| **raw** | XGBoost, CatBoost, TabPFN | Τα δεντρικά μοντέλα (Trees) και το TabPFN διαχειρίζονται καλά τα αρχικά, μη-κανονικοποιημένα δεδομένα. |
| **quantile** | TrueTabR | Τα Neural Networks (TabR) απαιτούν κανονική κατανομή εισόδου (Gaussian rank scaling) για σωστή σύγκλιση. |
| **pca/ica** | XGBoost, CatBoost | Τα δέντρα μπορούν να εκμεταλλευτούν τις ορθογώνιες συνιστώσες για να βρουν split points σε πλάγιους άξονες. |

**Ειδική Σημείωση για TabPFN:**
> Παρόλο που οι δημιουργοί του TabPFN προτείνουν τη χρήση "Raw Data" χωρίς προεπεξεργασία, πειραματικά διαπιστώσαμε ότι η απόδοση βελτιώνεται όταν τροφοδοτείται με το **Enhanced Pipeline** (Razor-filtered features + Manifold/DAE augmentations). Έτσι, στην υλοποίησή μας, το TabPFN λαμβάνει τα εμπλουτισμένα δεδομένα της `raw` view και όχι μόνο τα αρχικά columns.

In [ ]:
def predict_probs_for_view(view, seed, X_train_base, X_test_base, y_enc, num_classes,
                            pseudo_idx=None, pseudo_y=None, pseudo_w=None,
                            X_train_raw=None, X_test_raw=None, razor_masks=None):
    pseudo = normalize_pseudo(pseudo_idx=pseudo_idx, pseudo_y=pseudo_y, pseudo_w=pseudo_w)
    
    names_models = [
        ('XGB_DART', get_xgb_dart(num_classes, iterations=config.GBDT_ITERATIONS)),
        ('Cat_Langevin', get_cat_langevin(num_classes, iterations=config.GBDT_ITERATIONS * 2)),
        ('TrueTabR', TrueTabR(num_classes)),
    ]
    
    if config.USE_TABPFN:
        tabpfn = TabPFNWrapper(device='cuda' if torch.cuda.is_available() else 'cpu', n_estimators=config.TABPFN_N_ENSEMBLES)
        names_models.append(('TabPFN', tabpfn))

    # Smart View Routing
    active_models = []
    view_norm = view.strip().lower()
    
    if view_norm == 'quantile':
        for name, model in names_models:
            if name in ['TrueTabR']: active_models.append((name, model))
    elif view_norm == 'raw':
        for name, model in names_models:
            if name in ['XGB_DART', 'Cat_Langevin', 'TabPFN']: active_models.append((name, model))
    elif view_norm in ['pca', 'ica', 'rp']:
        for name, model in names_models:
            if name in ['XGB_DART', 'Cat_Langevin']: active_models.append((name, model))
    else:
        active_models = names_models
        
    if not active_models:
        active_models = names_models
    names_models = active_models

    if config.USE_STACKING:
        return fit_predict_stacking(
            names_models=names_models, view_name=view, X_train_base=X_train_base, X_test_base=X_test_base,
            y=y_enc, num_classes=num_classes, cv_splits=config.N_FOLDS, seed=seed,
            pseudo_idx=pseudo_idx, pseudo_y=pseudo_y, pseudo_w=pseudo_w,
        )
    
    # Fallback: Simple averaging
    X_v, X_test_v = apply_feature_view(X_train_base, X_test_base, view=view, seed=seed, allow_transductive=config.ALLOW_TRANSDUCTIVE)
    X_tree_tr, X_tree_te, X_neural_tr, X_neural_te, _, _ = build_streams(X_v, X_test_v)
    
    view_probs = 0
    for name, base in names_models:
        print(f"  > Calibrating {name} ({config.N_FOLDS}-Fold)...")
        data_tr = X_tree_tr if ('XGB' in name or 'Cat' in name) else X_neural_tr
        data_te = X_tree_te if ('XGB' in name or 'Cat' in name) else X_neural_te
        calibrated = CalibratedModel(base, name)
        calibrated.fit(data_tr, y_enc)
        view_probs += calibrated.predict_proba(data_te)
    return view_probs / len(names_models)

## 16. Main Orchestration (Κύρια Ενορχήστρωση)

**Σκοπός:** Ο κεντρικός συντονισμός της εκτέλεσης του Τελικού Μοντέλου (Start-to-End Pipeline).

**Ροή Εκτέλεσης (Execution Flow):**

```mermaid
graph TD
    A[1. Load & Preprocess Data] --> B[2. Razor Feature Selection]
    B --> C{Self-Training Loop}
    C -- Iteration 0 --> D[Train Ensemble (No Pseudo)]
    D --> E[Mine High-Confidence Pseudo-Labels]
    E --> C
    C -- Iteration 1 --> F[Retrain Ensemble (+Pseudo)]
    F --> G[4. Aggregate Predictions]
    G --> H[5 seeds × 4 views]
    H --> I[5. Final Output Generation]
```

1. **Load Data:** Φόρτωση raw δεδομένων.
2. **Razor Feature Selection:** Επιλογή σημαντικών χαρακτηριστικών με CatBoost/XGBoost importance (CV-averaged).
3. **Self-Training Loop:**
   - *Iteration 0:* Αρχική εκπαίδευση χωρίς pseudo-labels.
   - *Mining:* Εξόρυξη δειγμάτων test με υψηλό confidence (>0.95) και συμφωνία μεταξύ views.
   - *Iteration 1:* Επανεκπαίδευση όλων των μοντέλων συμπεριλαμβάνοντας τα pseudo-labels.
4. **Aggregation:** Συνδυασμός προβλέψεων από όλα τα seeds και τα views (Monte Carlo Ensemble).
5. **Save:** Αποθήκευση τελικών ετικετών `outputs/labelsX_grandmaster.npy`.

In [ ]:
def run_final_model():    """Κύρια συνάρτηση εκτέλεσης του Sigma-Omega Grandmaster Protocol."""    print(">>> INITIATING ΤΕΛΙΚΟ ΜΟΝΤΕΛΟ <<<")        # 1. Load Data    X, y, X_test = load_data()    le = LabelEncoder()    y_enc = le.fit_transform(y)    num_classes = len(le.classes_)    print(f"Classes: {num_classes}, Train: {X.shape}, Test: {X_test.shape}")        # 2. Razor (Feature Selection via CatBoost)    print("[RAZOR] Computing feature importance...")    from catboost import CatBoostClassifier    scout = CatBoostClassifier(iterations=500, verbose=0, task_type='GPU' if torch.cuda.is_available() else 'CPU', random_seed=42)    scout.fit(X, y_enc)    importance = scout.get_feature_importance()    razor_threshold = np.percentile(importance, 10)    keep_mask = importance > razor_threshold    X_razor = X[:, keep_mask]    X_test_razor = X_test[:, keep_mask]    print(f"  > Razor: {np.sum(keep_mask)}/{X.shape[1]} features kept")        # 3. Self-Training Loop    if config.ENABLE_SELF_TRAIN and config.SELF_TRAIN_ITERS > 0:        pseudo = PseudoData.empty()        last_avg_probs = None                for it in range(int(config.SELF_TRAIN_ITERS) + 1):            if it == 0:                print(f"\n>>> SELF-TRAIN ITERATION 0 (no pseudo) <<<")            else:                print(f"\n>>> SELF-TRAIN ITERATION {it} (pseudo={len(pseudo.idx)}) <<<")                        probs_per_view = {v: [] for v in VIEWS}            preds_per_view = {v: [] for v in VIEWS}                        for seed in config.SEEDS:                seed_everything(seed)                for view in VIEWS:                    p = predict_probs_for_view(                        view, seed, X_razor, X_test_razor, y_enc, num_classes,                        pseudo_idx=pseudo.idx, pseudo_y=pseudo.y, pseudo_w=pseudo.w,                        X_train_raw=X, X_test_raw=X_test,                    )                    probs_per_view[view].append(p)                    preds_per_view[view].append(np.argmax(p, axis=1))                        # Aggregate            probs_tensor = []            preds_tensor = []            for view in VIEWS:                probs_tensor.append(np.stack(probs_per_view[view], axis=0))                preds_tensor.append(np.stack(preds_per_view[view], axis=0))            probs_tensor = np.stack(probs_tensor, axis=0)  # (V, S, N, C)            preds_tensor = np.stack(preds_tensor, axis=0)  # (V, S, N)                        avg_probs = probs_tensor.mean(axis=(0, 1))            last_avg_probs = avg_probs                        # Mine pseudo-labels for next iteration            if it < int(config.SELF_TRAIN_ITERS):                votes = preds_tensor.reshape(-1, preds_tensor.shape[2])                mode_pred, agree_frac_votes = vote_mode_and_agreement(votes)                view_agree_frac = view_agreement_fraction(preds_tensor, mode_pred)                                conf = np.max(avg_probs, axis=1)                mask = (                    (conf >= float(config.SELF_TRAIN_CONF))                    & (agree_frac_votes >= float(config.SELF_TRAIN_AGREE))                    & (view_agree_frac >= float(config.SELF_TRAIN_VIEW_AGREE))                )                idx = np.nonzero(mask)[0]                                if idx.size > int(config.SELF_TRAIN_MAX):                    top = np.argsort(conf[idx])[::-1][:int(config.SELF_TRAIN_MAX)]                    idx = idx[top]                                pseudo_idx = idx.astype(np.int64)                pseudo_y = avg_probs[pseudo_idx]  # Soft labels                pseudo_w = np.power(conf[pseudo_idx].astype(np.float32), float(config.SELF_TRAIN_WEIGHT_POWER))                pseudo = PseudoData(idx=pseudo_idx, y=pseudo_y, w=pseudo_w)                                print(f"  [SELF-TRAIN] mined {len(pseudo_idx)} pseudo (conf>={config.SELF_TRAIN_CONF})")                final_ensemble_probs = last_avg_probs    else:        # Standard execution without self-training        final_ensemble_probs = 0        for seed in config.SEEDS:            print(f"\n>>> SEQUENCE START: SEED {seed} <<<")            seed_everything(seed)            view_probs_total = 0            for view in VIEWS:                print(f"  [VIEW] {view}")                view_probs_total += predict_probs_for_view(                    view, seed, X_razor, X_test_razor, y_enc, num_classes,                    X_train_raw=X, X_test_raw=X_test,                )            if isinstance(final_ensemble_probs, int):                final_ensemble_probs = view_probs_total / len(VIEWS)            else:                final_ensemble_probs += view_probs_total / len(VIEWS)        final_ensemble_probs /= len(config.SEEDS)        # 4. Final Output    preds = np.argmax(final_ensemble_probs, axis=1)    labels = le.inverse_transform(preds)        os.makedirs('outputs', exist_ok=True)    np.save('outputs/labelsX_final.npy', labels)    print("\n>>> ΤΕΛΙΚΟ ΜΟΝΤΕΛΟ ΟΛΟΚΛΗΡΩΘΗΚΕ <<<")    print(f"Predictions saved to: outputs/labelsX_final.npy")    return labels, final_ensemble_probs

## 17. Εκτέλεση

**Σκοπός:** Entry point για εκτέλεση του pipeline.

**Προσοχή:** 
- Απαιτεί GPU με ≥8GB VRAM
- Χρόνος εκτέλεσης: ~1-3 ώρες
- Αποθηκεύει checkpoints

In [ ]:
# Εκτελέστε αυτό το κελί για να τρέξετε το πλήρες pipeline
# ΠΡΟΣΟΧΗ: Απαιτεί σημαντικό χρόνο (~1-3 ώρες ανάλογα με το hardware)

if __name__ == '__main__':
    # Uncomment την παρακάτω γραμμή για να τρέξετε το pipeline
    # labels, probs = run_grandmaster_protocol()
    print("Αποσχολιάστε την γραμμή παραπάνω για να εκτελέσετε το pipeline")

## 18. Επαλήθευση Αποτελεσμάτων

**Σκοπός:** Φόρτωση και επαλήθευση των predictions.

**Έλεγχοι:**
- Αριθμός predictions = αριθμός test samples
- Class distribution

In [ ]:
# Φόρτωση αποτελεσμάτων αν υπάρχουν
output_path = 'outputs/labelsX_grandmaster.npy'
if os.path.exists(output_path):
    labels = np.load(output_path, allow_pickle=True)
    print(f"Loaded predictions: {len(labels)} samples")
    print(f"Class distribution: {np.unique(labels, return_counts=True)}")
else:
    # Try parent directory
    alt_path = 'PartD/outputs/labelsX_grandmaster.npy'
    if os.path.exists(alt_path):
        labels = np.load(alt_path, allow_pickle=True)
        print(f"Loaded predictions from {alt_path}: {len(labels)} samples")
        print(f"Class distribution: {np.unique(labels, return_counts=True)}")
    else:
        print("No predictions found. Run the pipeline first.")